# HSE Машинное обучение и майнинг данных

## Семинар 6 Часть 1: Задачи регрессии

### Полина Полунина
polina.polunina@skolkovotech.ru

# Стекинг, вотинг, градиентный бустинг


Import packages:

In [1]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
#ml models
import scipy as sp
import sklearn
from sklearn import datasets
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.svm import SVR
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
#linear regression
import statsmodels.api as sm
#set style for plots
sns.set_style('darkgrid')
#off the warnings
import warnings
warnings.filterwarnings("ignore")

## Часть 1: Для задачи регрессии:

* подгрузите данные "train.csv" 
* поделите данные на тренировочную и тестовую выборки
* обучите: линейную регрессию, svm, random forest
* используйте предсказания от этих трех моделей для проверки качества ансамбля
* используя предложенные техники ансамблей, проверьте, улучшился ли результат

### Решение

*Подгружаем данные*

In [2]:
cols=['Id', 'MSSubClass', 'LotArea', 'OverallQual',\
      'OverallCond', 'YearBuilt', 'YearRemodAdd',\
      'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF',\
     'LowQualFinSF', 'GrLivArea', 'BsmtFullBath',\
     'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr',\
     'TotRmsAbvGrd', 'Fireplaces', 'GarageCars',\
     'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch',\
     'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold', 'SalePrice']

In [3]:
data = pd.read_csv('train.csv', index_col=0, usecols=cols)
data.head()

,MSSubClass,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,...,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,8450,7,5,2003,2003,150,856,856,854,...,0,61,0,0,0,0,0,2,2008,208500
2,20,9600,6,8,1976,1976,284,1262,1262,0,...,298,0,0,0,0,0,0,5,2007,181500
3,60,11250,7,5,2001,2002,434,920,920,866,...,0,42,0,0,0,0,0,9,2008,223500
4,70,9550,7,5,1915,1970,540,756,961,756,...,0,35,272,0,0,0,0,2,2006,140000
5,60,14260,8,5,2000,2000,490,1145,1145,1053,...,192,84,0,0,0,0,0,12,2008,250000


*Делим данные на тренировочную и тестовую выборки*

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), data.SalePrice, test_size=0.2, random_state=32)

*Averaging*

Алгоритм: 
* обучаем 3 модели на тренировочной выборке
* делаем 3 предикта на тестовой выборке
* усредняем
* считаем целевую метрику $R^2$


In [6]:
# модель 1: линейная регрессия
from sklearn.linear_model import LinearRegression
# определяем и обучаем модель на тренировочной выборке
linreg = LinearRegression().fit(X_train, y_train)
# предсказываем на тестовой выборке
preds_linreg = linreg.predict(X_test)
# считаем скор на тестовой выборке
linreg.score(X_test, y_test)

0.7638868743142179

In [7]:
# модель 2: random forest
from sklearn.ensemble import RandomForestRegressor
# определяем модель
rf = sklearn.ensemble.RandomForestRegressor()
# обучаем модель
model_rf = rf.fit(X_train, y_train)
# предсказываем
preds_rf = model_rf.predict(X_test)
# считаем скор
model_rf.score(X_test, y_test)

0.7732069639497671

In [8]:
# модель 3: svm
from sklearn.svm import SVR
# определяем модель
svm = SVR(kernel='linear')
# обучаем модель
model_svm = svm.fit(X_train, y_train)
# предсказываем
preds_svm = model_svm.predict(X_test)
# считаем скор
model_svm.score(X_test, y_test)

0.698663801279436

In [9]:
# усредняем три предсказания
averaged_preds = (preds_linreg + preds_rf + preds_svm) / 3

In [10]:
# считаем скор от усреднения
from sklearn.metrics import r2_score
r2_score(y_test, averaged_preds)

0.7772996829256635

In [12]:
# считаем взвешенное среднее
averaged_weighted_preds = 0.2 * preds_linreg + 0.7 * preds_rf + 0.1 * preds_svm
# считаем скор
r2_score(y_test, averaged_weighted_preds)

0.7864826070980142

*Stacking*

Алгоритм:
* обучаем 3 модели
* делаем предсказания на тренировочной и тестовой выборках
* добавляем предсказания от трех моделей в качестве новых признаков в X_train и X_test
* обучаемся любой новой моделью на дополненном тренировочном сете
* предсказываем на дополненной тестовой выборке
* считаем скор

In [13]:
# предсказания на тренировочной выборке для трех моделей
# модель 1
preds_linreg_train = linreg.predict(X_train)
# модель 2
preds_rf_train = model_rf.predict(X_train)
# модель 3
preds_svm_train = model_svm.predict(X_train)

In [14]:
# добавляем предсказания в качестве новых признаков в тренировочную и тестовую выборки
X_train['linreg'] = preds_linreg_train
X_test['linreg'] = preds_linreg
X_train['rf'] = preds_rf_train
X_test['rf'] = preds_rf
X_train['svm'] = preds_svm_train
X_test['svm'] = preds_svm

In [23]:
# обучаем новую модель на дополненной тренировочной выборке
new_model_rf = rf.fit(X_train, y_train)
# считаем скор
new_model_rf.score(X_test, y_test)

0.7775139962784351

*Blending*

Алгоритм:
 * делим тренировочный сет на две части
 * обучаемся тремя моделями на первой части
 * делаем предсказания для второй части и тестовой выборки
 * добавляем предсказания для второй части и тестовой выборки в эту самую вторую часть и тестовую выборку
 * обучаемся новой моделью на дополненной второй части
 * делаем предсказания для дополненной тестовой выборки
 * считаем скор
 

In [24]:
# убираем добавленные на этапе стекинга признаки
X_train = X_train.drop(['linreg', 'rf', 'svm'], axis=1)
X_test = X_test.drop(['linreg', 'rf', 'svm'], axis=1)

In [25]:
# делим тренировочный сет на две части
X_train_1, X_train_2, y_train_1, y_train_2 = train_test_split(X_train, y_train, test_size=0.4, random_state=32)

In [26]:
# обучаемся тремя моделями на первой части тренировочного сета
# модель 1
linreg = LinearRegression().fit(X_train_1, y_train_1)
preds_linreg_2 = linreg.predict(X_train_2)
preds_linreg_test = linreg.predict(X_test)
# скор на тесте
linreg.score(X_test, y_test)

0.7549490198212518

In [27]:
# модель 2
rf = sklearn.ensemble.RandomForestRegressor()
model_rf = rf.fit(X_train_1, y_train_1)
preds_rf_2 = model_rf.predict(X_train_2)
preds_rf_test = model_rf.predict(X_test)
# считаем скор на тесте
model_rf.score(X_test, y_test)

0.7669397491336264

In [28]:
# модель 3
svm = SVR(kernel='linear')
model_svm = svm.fit(X_train_1, y_train_1)
preds_svm_2 = model_svm.predict(X_train_2)
preds_svm_test = model_svm.predict(X_test)
# считаем скор на тесте
model_svm.score(X_test, y_test)

0.687485402687889

In [29]:
# добавляем признаки во вторую часть тренировочной и тестовую выборки
X_train_2['linreg'] = preds_linreg_2
X_test['linreg'] = preds_linreg_test
X_train_2['rf'] = preds_rf_2
X_test['rf'] = preds_rf_test
X_train_2['svm'] = preds_svm_2
X_test['svm'] = preds_svm_test

In [32]:
# обучаемся новой моделью на дополненной второй части тренировочной выборки
new_model_rf = rf.fit(X_train_2, y_train_2)
# считаем скор
new_model_rf.score(X_test, y_test)

0.8100121321509063

*Bagging*

Алгоритм:
* делим тренировочный датасет на N частей с возвращением (бутстреппинг)
* обучаем на каждой части разные модели и делаем предикт на тестовой выборке
* финальное предсказание - усреднение всех предсказаний
* считаем скор

In [33]:
# убираем из тестовой выборки добавленные при блендинге фичи
X_test = X_test.drop(['linreg', 'rf', 'svm'], axis=1)

In [34]:
from sklearn.tree import DecisionTreeRegressor

In [36]:
# вспомогательный массив для предсказаний различных моделей
preds = []

# число моделей
n = 40
# кол-во строк в подвыборках
m = 300

# цикл для обучения моделей
for i in range(n):
    # подвыборка тренировочного сета: генерируем индексы строк случайным образом
    idx = np.random.choice(X_train.shape[0], m, replace=False)
    # подвыборка тренировочного сета: находим подвыборку
    X_sample = X_train.iloc[idx, :]
    y_sample = y_train.iloc[idx]
    # определяем модель, обучаем, делаем предсказания
    reg = DecisionTreeRegressor()
    model = reg.fit(X_sample, y_sample)
    predictions = model.predict(X_test)
    # сохраняем предсказания в наш вспомогательный массив
    preds.append(predictions)
    # выводим на экран индивидуальные скоры на тесте
    print('выборка {} | R2 на тесте: {:.2f}'.format(i+1, model.score(X_test, y_test)))
    
# финальное предсказание - усреднение всех предыдущих
final_preds = np.mean(preds, axis=0)
# считаем скор
print('\n\nфинальный скор на тесте: {:.2f}'.format(r2_score(y_test, final_preds)))

выборка 1 | R2 на тесте: 0.68
выборка 2 | R2 на тесте: 0.68
выборка 3 | R2 на тесте: 0.55
выборка 4 | R2 на тесте: 0.58
выборка 5 | R2 на тесте: 0.73
выборка 6 | R2 на тесте: 0.67
выборка 7 | R2 на тесте: 0.60
выборка 8 | R2 на тесте: 0.64
выборка 9 | R2 на тесте: 0.57
выборка 10 | R2 на тесте: 0.56
выборка 11 | R2 на тесте: 0.65
выборка 12 | R2 на тесте: 0.63
выборка 13 | R2 на тесте: 0.59
выборка 14 | R2 на тесте: 0.50
выборка 15 | R2 на тесте: 0.59
выборка 16 | R2 на тесте: 0.67
выборка 17 | R2 на тесте: 0.61
выборка 18 | R2 на тесте: 0.43
выборка 19 | R2 на тесте: 0.61
выборка 20 | R2 на тесте: 0.69
выборка 21 | R2 на тесте: 0.70
выборка 22 | R2 на тесте: 0.70
выборка 23 | R2 на тесте: 0.47
выборка 24 | R2 на тесте: 0.67
выборка 25 | R2 на тесте: 0.64
выборка 26 | R2 на тесте: 0.66
выборка 27 | R2 на тесте: 0.69
выборка 28 | R2 на тесте: 0.54
выборка 29 | R2 на тесте: 0.40
выборка 30 | R2 на тесте: 0.62
выборка 31 | R2 на тесте: 0.31
выборка 32 | R2 на тесте: 0.69
выборка 33 | R2 н

## Часть 2: Градиентный бустинг с пакетом catboost

На этом этапе мы будем использовать полный набор данных, но сначала уберем все признаки с большим кол-вом пропусков в данных, а затем уберем все оставшиеся строки с пропусками

In [37]:
# подгружаем полный набор данных
data = pd.read_csv('train.csv', index_col=0)
data.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [38]:
# удаляем признаки с кол-вом пропусков больше 200
mask = (data.isna().sum() <= 200)
cols = data.isna().sum()[mask].index.tolist()
data = data[cols]
# удаляем оставшиеся строки с пропусками 
data.dropna(inplace=True)
cat_features = data.dtypes[data.dtypes == 'object'].index.tolist()

Пакет catboost умеет работать с категориальными признаками, для этого сохраним названия столбцов с категориальными данными в отдельный массив

In [39]:
cat_features = data.dtypes[data.dtypes == 'object'].index.tolist()

Разделим данные на тренировочную, валидационную и тестовую выборки

In [41]:
# делим данные первый раз на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(data.drop('SalePrice', axis=1), data.SalePrice,\
                                                    test_size=0.2, random_state=32)
# делим тренировочный кусок на трейн и валидацию
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

Импортируем catboost

In [42]:
from catboost import CatBoostRegressor, Pool

Используем Pool конструктор, чтобы создать тренировочный, валидационный и тестовый сеты. Передаем в соответствующую переменную имена категориальных столбцов.

In [43]:
train_data = Pool(X_train, y_train, cat_features=cat_features)
val_data = Pool(X_val, y_val, cat_features=cat_features)
test_data = Pool(X_test, y_test, cat_features=cat_features)

Прописываем желаемые параметры для алгоритма:
* iterations - кол-во итераций
* learning_rate - параметр, используется для варьирования шага градиента; чем значение ниже, тем медленнее сходится алгоритм
* depth - глубина деревьев, оптимальные значения от 4 до 10
* eval_metric - желаемая метрика для мониторинга качества модели
* verbose - параметр, отвечающий за то, выводить ли результаты каждой итерации на экран

Ссылка на описание всех возможных параметров для тюнинга модели: https://catboost.ai/docs/concepts/parameter-tuning.html

Обратите внимание, что learning rate можно формально не прописывать, тогда он будет подобран автоматический в зависимости от кол-ва итераций

In [44]:
params={'iterations' : 500,
        'learning_rate' : 0.03,
        'eval_metric' : 'R2',
        'depth' : 5,
        'verbose' : True
        }

Далее все выглядит привычным нам образом

In [45]:
# определяем модель
model = CatBoostRegressor(**params)

In [46]:
# обучаем модель на тренировочных данных, при этом на каждой итерации мониторим результат на валидационной выборке
model.fit(train_data, eval_set=val_data)

0:	learn: 0.0034681	test: 0.0041094	best: 0.0041094 (0)	total: 569ms	remaining: 4m 43s
1:	learn: 0.0088492	test: 0.0091889	best: 0.0091889 (1)	total: 855ms	remaining: 3m 32s
2:	learn: 0.0109156	test: 0.0124937	best: 0.0124937 (2)	total: 1.1s	remaining: 3m 2s
3:	learn: 0.0154288	test: 0.0177270	best: 0.0177270 (3)	total: 1.36s	remaining: 2m 48s
4:	learn: 0.0166346	test: 0.0190894	best: 0.0190894 (4)	total: 1.51s	remaining: 2m 29s
5:	learn: 0.0214098	test: 0.0234166	best: 0.0234166 (5)	total: 1.8s	remaining: 2m 28s
6:	learn: 0.0270514	test: 0.0291991	best: 0.0291991 (6)	total: 2.05s	remaining: 2m 24s
7:	learn: 0.0337481	test: 0.0352915	best: 0.0352915 (7)	total: 2.22s	remaining: 2m 16s
8:	learn: 0.0411919	test: 0.0419029	best: 0.0419029 (8)	total: 2.39s	remaining: 2m 10s
9:	learn: 0.0446383	test: 0.0450045	best: 0.0450045 (9)	total: 2.65s	remaining: 2m 9s
10:	learn: 0.0500254	test: 0.0505311	best: 0.0505311 (10)	total: 2.89s	remaining: 2m 8s
11:	learn: 0.0574758	test: 0.0583037	best: 0.0

93:	learn: 0.7845765	test: 0.7151288	best: 0.7151288 (93)	total: 18.3s	remaining: 1m 18s
94:	learn: 0.7865940	test: 0.7168996	best: 0.7168996 (94)	total: 18.4s	remaining: 1m 18s
95:	learn: 0.7898901	test: 0.7201620	best: 0.7201620 (95)	total: 18.6s	remaining: 1m 18s
96:	learn: 0.7927153	test: 0.7225165	best: 0.7225165 (96)	total: 18.8s	remaining: 1m 18s
97:	learn: 0.7954766	test: 0.7246201	best: 0.7246201 (97)	total: 19s	remaining: 1m 18s
98:	learn: 0.7983017	test: 0.7276938	best: 0.7276938 (98)	total: 19.2s	remaining: 1m 17s
99:	learn: 0.8006621	test: 0.7298563	best: 0.7298563 (99)	total: 19.4s	remaining: 1m 17s
100:	learn: 0.8025283	test: 0.7312756	best: 0.7312756 (100)	total: 19.5s	remaining: 1m 17s
101:	learn: 0.8051539	test: 0.7341919	best: 0.7341919 (101)	total: 19.8s	remaining: 1m 17s
102:	learn: 0.8069293	test: 0.7357171	best: 0.7357171 (102)	total: 20s	remaining: 1m 16s
103:	learn: 0.8094840	test: 0.7373863	best: 0.7373863 (103)	total: 20.1s	remaining: 1m 16s
104:	learn: 0.812

185:	learn: 0.8843161	test: 0.7884512	best: 0.7884512 (185)	total: 37.8s	remaining: 1m 3s
186:	learn: 0.8845928	test: 0.7884729	best: 0.7884729 (186)	total: 38s	remaining: 1m 3s
187:	learn: 0.8849047	test: 0.7886407	best: 0.7886407 (187)	total: 38.1s	remaining: 1m 3s
188:	learn: 0.8849264	test: 0.7886241	best: 0.7886407 (187)	total: 38.2s	remaining: 1m 2s
189:	learn: 0.8853567	test: 0.7889177	best: 0.7889177 (189)	total: 38.3s	remaining: 1m 2s
190:	learn: 0.8855402	test: 0.7892584	best: 0.7892584 (190)	total: 38.4s	remaining: 1m 2s
191:	learn: 0.8856125	test: 0.7893258	best: 0.7893258 (191)	total: 38.5s	remaining: 1m 1s
192:	learn: 0.8861691	test: 0.7890619	best: 0.7893258 (191)	total: 38.7s	remaining: 1m 1s
193:	learn: 0.8866220	test: 0.7892890	best: 0.7893258 (191)	total: 38.9s	remaining: 1m 1s
194:	learn: 0.8868599	test: 0.7895230	best: 0.7895230 (194)	total: 39.1s	remaining: 1m 1s
195:	learn: 0.8870914	test: 0.7891038	best: 0.7895230 (194)	total: 39.4s	remaining: 1m 1s
196:	learn: 

277:	learn: 0.9035764	test: 0.8043703	best: 0.8043703 (277)	total: 55.1s	remaining: 44s
278:	learn: 0.9037968	test: 0.8044103	best: 0.8044103 (278)	total: 55.3s	remaining: 43.8s
279:	learn: 0.9039955	test: 0.8045526	best: 0.8045526 (279)	total: 55.6s	remaining: 43.6s
280:	learn: 0.9040525	test: 0.8046386	best: 0.8046386 (280)	total: 55.7s	remaining: 43.4s
281:	learn: 0.9041128	test: 0.8049478	best: 0.8049478 (281)	total: 55.8s	remaining: 43.1s
282:	learn: 0.9042734	test: 0.8054051	best: 0.8054051 (282)	total: 55.9s	remaining: 42.9s
283:	learn: 0.9044278	test: 0.8059137	best: 0.8059137 (283)	total: 56.1s	remaining: 42.7s
284:	learn: 0.9044564	test: 0.8059749	best: 0.8059749 (284)	total: 56.2s	remaining: 42.4s
285:	learn: 0.9045486	test: 0.8061785	best: 0.8061785 (285)	total: 56.4s	remaining: 42.2s
286:	learn: 0.9046920	test: 0.8061899	best: 0.8061899 (286)	total: 56.6s	remaining: 42s
287:	learn: 0.9047392	test: 0.8063960	best: 0.8063960 (287)	total: 56.7s	remaining: 41.7s
288:	learn: 0.

369:	learn: 0.9152591	test: 0.8165187	best: 0.8165187 (369)	total: 1m 14s	remaining: 26.2s
370:	learn: 0.9154149	test: 0.8168369	best: 0.8168369 (370)	total: 1m 14s	remaining: 26s
371:	learn: 0.9155472	test: 0.8168927	best: 0.8168927 (371)	total: 1m 14s	remaining: 25.8s
372:	learn: 0.9156161	test: 0.8168802	best: 0.8168927 (371)	total: 1m 15s	remaining: 25.6s
373:	learn: 0.9156705	test: 0.8169356	best: 0.8169356 (373)	total: 1m 15s	remaining: 25.4s
374:	learn: 0.9156895	test: 0.8170765	best: 0.8170765 (374)	total: 1m 15s	remaining: 25.2s
375:	learn: 0.9157533	test: 0.8171585	best: 0.8171585 (375)	total: 1m 15s	remaining: 25s
376:	learn: 0.9158023	test: 0.8171733	best: 0.8171733 (376)	total: 1m 16s	remaining: 24.8s
377:	learn: 0.9158759	test: 0.8171764	best: 0.8171764 (377)	total: 1m 16s	remaining: 24.6s
378:	learn: 0.9160505	test: 0.8170686	best: 0.8171764 (377)	total: 1m 16s	remaining: 24.4s
379:	learn: 0.9160975	test: 0.8172718	best: 0.8172718 (379)	total: 1m 16s	remaining: 24.2s
380

461:	learn: 0.9221984	test: 0.8206486	best: 0.8206486 (461)	total: 1m 34s	remaining: 7.76s
462:	learn: 0.9222178	test: 0.8207119	best: 0.8207119 (462)	total: 1m 34s	remaining: 7.55s
463:	learn: 0.9222508	test: 0.8206848	best: 0.8207119 (462)	total: 1m 34s	remaining: 7.35s
464:	learn: 0.9222652	test: 0.8207802	best: 0.8207802 (464)	total: 1m 34s	remaining: 7.15s
465:	learn: 0.9223613	test: 0.8208463	best: 0.8208463 (465)	total: 1m 35s	remaining: 6.95s
466:	learn: 0.9223986	test: 0.8207892	best: 0.8208463 (465)	total: 1m 35s	remaining: 6.75s
467:	learn: 0.9225474	test: 0.8209701	best: 0.8209701 (467)	total: 1m 35s	remaining: 6.54s
468:	learn: 0.9225815	test: 0.8209608	best: 0.8209701 (467)	total: 1m 35s	remaining: 6.34s
469:	learn: 0.9226176	test: 0.8209032	best: 0.8209701 (467)	total: 1m 36s	remaining: 6.14s
470:	learn: 0.9226462	test: 0.8208023	best: 0.8209701 (467)	total: 1m 36s	remaining: 5.93s
471:	learn: 0.9227986	test: 0.8211031	best: 0.8211031 (471)	total: 1m 36s	remaining: 5.73s

Что мы видим на экране выше? 

* в первой колонке значение выбранной метрики, в нашей случае - $R^2$, на тренировочной выборке
* во второй колонке - на валидационной выборке
* в третьей колонке мы видим лучший результат на валидации за все обучение и номер соответствующей итерации
* в четвертой колонке общее кол-во потраченного времени на обучение
* в пятой колонке оставшее на обучение время

Что делать дальше?

* Обучаем модель с разными параметрами (пока не надоест) пока не добьемся наилучшего результата на валидационной выборке
* Далее, смотрим на какой итерации был наилучший результат и обучаемся на объединение тренировочной и валидационной выборки с ограничением до этой итерации
* Делаем предикт на тестовой выборке
* Считаем скор (если настоящие значения целевой переменной доступны)

In [47]:
# объединяем тренировочную и валидационную выборки в один сет 
X_train_new = pd.concat([X_train, X_val], axis=0)
y_train_new = pd.concat([y_train, y_val], axis=0)

In [48]:
# используем конструктор Pool
train_data = Pool(X_train_new, y_train_new, cat_features=cat_features)

In [49]:
# задаем параметры модели: те же, что и для наилучшей выбранной модели, но с урезанным кол-вом итерации до наилучшей итерации по выбранной модели
params={'iterations' : 494,
        'learning_rate' : 0.03,
        'eval_metric' : 'R2',
        'depth' : 5,
        'verbose' : True
        }

In [50]:
# определяем и обучаем модель
model = CatBoostRegressor(**params)
model.fit(train_data)

0:	learn: 0.0049599	total: 155ms	remaining: 1m 16s
1:	learn: 0.0101873	total: 254ms	remaining: 1m 2s
2:	learn: 0.0148849	total: 279ms	remaining: 45.6s
3:	learn: 0.0190256	total: 393ms	remaining: 48.2s
4:	learn: 0.0239074	total: 473ms	remaining: 46.3s
5:	learn: 0.0268328	total: 564ms	remaining: 45.9s
6:	learn: 0.0331416	total: 626ms	remaining: 43.6s
7:	learn: 0.0400743	total: 689ms	remaining: 41.9s
8:	learn: 0.0479129	total: 752ms	remaining: 40.5s
9:	learn: 0.0530142	total: 877ms	remaining: 42.4s
10:	learn: 0.0600833	total: 1.02s	remaining: 44.9s
11:	learn: 0.0684455	total: 1.13s	remaining: 45.2s
12:	learn: 0.0750073	total: 1.2s	remaining: 44.3s
13:	learn: 0.0827101	total: 1.28s	remaining: 43.8s
14:	learn: 0.0908058	total: 1.36s	remaining: 43.5s
15:	learn: 0.1003716	total: 1.46s	remaining: 43.5s
16:	learn: 0.1111325	total: 1.53s	remaining: 42.9s
17:	learn: 0.1191976	total: 1.55s	remaining: 41.1s
18:	learn: 0.1275839	total: 1.63s	remaining: 40.7s
19:	learn: 0.1368931	total: 1.81s	remaini

163:	learn: 0.8910221	total: 17.6s	remaining: 35.4s
164:	learn: 0.8912501	total: 17.9s	remaining: 35.7s
165:	learn: 0.8917952	total: 18.3s	remaining: 36.2s
166:	learn: 0.8922831	total: 18.5s	remaining: 36.2s
167:	learn: 0.8925955	total: 18.6s	remaining: 36s
168:	learn: 0.8931767	total: 18.6s	remaining: 35.8s
169:	learn: 0.8934328	total: 18.7s	remaining: 35.7s
170:	learn: 0.8940868	total: 18.8s	remaining: 35.5s
171:	learn: 0.8946742	total: 18.8s	remaining: 35.3s
172:	learn: 0.8953715	total: 18.9s	remaining: 35.1s
173:	learn: 0.8960515	total: 19s	remaining: 35s
174:	learn: 0.8964706	total: 19.1s	remaining: 34.8s
175:	learn: 0.8970873	total: 19.2s	remaining: 34.6s
176:	learn: 0.8973259	total: 19.2s	remaining: 34.4s
177:	learn: 0.8978549	total: 19.5s	remaining: 34.5s
178:	learn: 0.8983445	total: 20s	remaining: 35.2s
179:	learn: 0.8990829	total: 20.4s	remaining: 35.6s
180:	learn: 0.8995779	total: 20.5s	remaining: 35.4s
181:	learn: 0.9000854	total: 20.5s	remaining: 35.2s
182:	learn: 0.900442

322:	learn: 0.9371933	total: 35s	remaining: 18.5s
323:	learn: 0.9373308	total: 35.3s	remaining: 18.5s
324:	learn: 0.9375152	total: 35.4s	remaining: 18.4s
325:	learn: 0.9375843	total: 36.1s	remaining: 18.6s
326:	learn: 0.9377087	total: 36.5s	remaining: 18.7s
327:	learn: 0.9378323	total: 36.6s	remaining: 18.5s
328:	learn: 0.9379834	total: 36.7s	remaining: 18.4s
329:	learn: 0.9381406	total: 36.8s	remaining: 18.3s
330:	learn: 0.9382423	total: 36.8s	remaining: 18.1s
331:	learn: 0.9383493	total: 36.9s	remaining: 18s
332:	learn: 0.9385032	total: 37s	remaining: 17.9s
333:	learn: 0.9385690	total: 37s	remaining: 17.7s
334:	learn: 0.9387675	total: 37.1s	remaining: 17.6s
335:	learn: 0.9389740	total: 37.2s	remaining: 17.5s
336:	learn: 0.9390686	total: 37.3s	remaining: 17.4s
337:	learn: 0.9391508	total: 37.4s	remaining: 17.3s
338:	learn: 0.9393531	total: 37.5s	remaining: 17.2s
339:	learn: 0.9394518	total: 37.9s	remaining: 17.1s
340:	learn: 0.9396507	total: 38s	remaining: 17s
341:	learn: 0.9398344	to

482:	learn: 0.9532486	total: 52.3s	remaining: 1.19s
483:	learn: 0.9533432	total: 52.4s	remaining: 1.08s
484:	learn: 0.9533922	total: 52.5s	remaining: 974ms
485:	learn: 0.9534497	total: 52.5s	remaining: 865ms
486:	learn: 0.9535383	total: 52.6s	remaining: 756ms
487:	learn: 0.9536154	total: 52.7s	remaining: 648ms
488:	learn: 0.9536964	total: 52.8s	remaining: 540ms
489:	learn: 0.9537231	total: 52.8s	remaining: 431ms
490:	learn: 0.9538004	total: 52.9s	remaining: 323ms
491:	learn: 0.9538784	total: 53s	remaining: 215ms
492:	learn: 0.9539518	total: 53.1s	remaining: 108ms
493:	learn: 0.9540180	total: 53.1s	remaining: 0us


In [51]:
# получаем предсказания на тестовых данных
preds = model.predict(test_data)

In [52]:
# считаем скор
r2_score(y_test, preds)

0.90477048769169